# 01_image_conversion.ipynb
## Pruebas de Conversión Excel/PDF a Imágenes

Objetivo: Prototipar y validar conversión de Excel y PDF a imágenes PNG

In [ ]:
import sys
from pathlib import Path

# Agregar src al path
sys.path.insert(0, str(Path.cwd()))

from src.preprocessors import excel_to_image, pdf_to_images
from src.utils.logger import get_logger
from src.utils.config import INPUT_EXCEL_DIR, INPUT_PDF_DIR, EXCEL_IMAGES_DIR, PDF_IMAGES_DIR

logger = get_logger(__name__)

## 1. Prueba: Excel a Imagen

In [ ]:
# Listar archivos Excel disponibles
excel_files = list(INPUT_EXCEL_DIR.glob("*.xlsx"))
print(f"Archivos Excel encontrados: {len(excel_files)}")
for f in excel_files:
    print(f"  - {f.name}")

In [ ]:
# Procesar primer Excel si existe
if excel_files:
    excel_file = excel_files[0]
    print(f"\nProcesando: {excel_file.name}")
    
    result = excel_to_image(str(excel_file))
    print(f"Resultado: {result['status']}")
    
    if result['status'] == 'success':
        print(f"HTML generado (primeros 200 chars):")
        print(result['html'][:200])
else:
    print("No hay archivos Excel en data/input/excel/")

## 2. Prueba: PDF a Imágenes

In [ ]:
# Listar archivos PDF disponibles
pdf_files = list(INPUT_PDF_DIR.glob("*.pdf"))
print(f"Archivos PDF encontrados: {len(pdf_files)}")
for f in pdf_files:
    print(f"  - {f.name}")

In [ ]:
# Obtener info del PDF
if pdf_files:
    pdf_file = pdf_files[0]
    print(f"\nAnalizando: {pdf_file.name}")
    
    from src.preprocessors import get_pdf_info
    info = get_pdf_info(str(pdf_file))
    print(f"Total de páginas: {info.get('total_pages', 'N/A')}")

In [ ]:
# Convertir PDF a imágenes (primeras 3 páginas)
if pdf_files:
    pdf_file = pdf_files[0]
    print(f"Convirtiendo PDF: {pdf_file.name}")
    
    # Solo primeras 3 páginas para testing
    result = pdf_to_images(str(pdf_file), dpi=150, page_range=[1, 3])
    
    print(f"Resultado: {result['status']}")
    print(f"Imágenes generadas: {result.get('image_count', 0)}")
    
    if result['status'] == 'success':
        print(f"Archivos: {result['metadata']['image_files']}")

## 3. Validación: Verificar Imágenes Generadas

In [ ]:
# Listar imágenes generadas
excel_images = list(EXCEL_IMAGES_DIR.glob("*.png"))
pdf_images = list(PDF_IMAGES_DIR.glob("*.png"))

print(f"Imágenes de Excel: {len(excel_images)}")
print(f"Imágenes de PDF: {len(pdf_images)}")
print(f"Total: {len(excel_images) + len(pdf_images)}")

In [ ]:
# Mostrar propiedades de imágenes
from PIL import Image

print("\nPropiedades de imágenes:")
all_images = excel_images + pdf_images

for img_path in all_images[:5]:  # Primeras 5
    img = Image.open(img_path)
    print(f"\n{img_path.name}:")
    print(f"  - Tamaño: {img.size}")
    print(f"  - Modo: {img.mode}")
    print(f"  - Bytes: {img_path.stat().st_size / 1024:.1f} KB")

## 4. Visualización (si disponible)

In [ ]:
# Mostrar imágenes
import matplotlib.pyplot as plt

if all_images:
    # Mostrar primeras 3 imágenes
    fig, axes = plt.subplots(1, min(3, len(all_images)), figsize=(15, 5))
    
    if len(all_images) == 1:
        axes = [axes]
    elif len(all_images) == 2:
        axes = [axes[0], axes[1]]
    
    for idx, img_path in enumerate(all_images[:3]):
        img = Image.open(img_path)
        axes[idx].imshow(img)
        axes[idx].set_title(img_path.name)
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.savefig('image_conversion_preview.png', dpi=100, bbox_inches='tight')
    plt.show()
    print("✅ Visualización guardada en image_conversion_preview.png")
else:
    print("No hay imágenes para visualizar")

## 5. Comparación: Diferentes Resoluciones

In [ ]:
# Comparar DPI (Quality vs Size)
if pdf_files:
    pdf_file = pdf_files[0]
    
    dpi_levels = [100, 150, 200, 300]
    results = {}
    
    for dpi in dpi_levels:
        print(f"\nProbando DPI: {dpi}")
        result = pdf_to_images(str(pdf_file), dpi=dpi, page_range=[1, 1])
        
        if result['status'] == 'success':
            # Obtener tamaño del archivo
            images = list(PDF_IMAGES_DIR.glob(f"*{dpi}*")) if dpi != 300 else list(PDF_IMAGES_DIR.glob("*.png"))
            if images:
                size_kb = images[0].stat().st_size / 1024
                results[dpi] = size_kb
                print(f"  ✅ Tamaño: {size_kb:.1f} KB")
    
    print(f"\nComparación DPI:")
    for dpi, size in results.items():
        print(f"  DPI {dpi}: {size:.1f} KB")

## 6. Resumen y Estadísticas

In [ ]:
print("="*60)
print("RESUMEN: CONVERSIÓN A IMÁGENES")
print("="*60)
print(f"\nExcel procesados: {len(excel_files)}")
print(f"PDF procesados: {len(pdf_files)}")
print(f"\nImágenes generadas:")
print(f"  - De Excel: {len(excel_images)}")
print(f"  - De PDF: {len(pdf_images)}")
print(f"  - Total: {len(excel_images) + len(pdf_images)}")
print(f"\nEstructura listo para:")
print(f"  ✅ OCR Extraction (siguiente notebook)")
print(f"  ✅ CLIP Encoding (notebook 03)")
print(f"  ✅ ChromaDB Indexing (notebook 04)")
print("="*60)